In [ ]:
import xarray as xr

from regridding import regrid

In [ ]:
ds_of160_grid = xr.open_dataset('/cluster/projects/nn9297k/OF160/Grid/OF160_grid_v1.nc')
ds_of800_clim = xr.open_dataset('/cluster/projects/nn9297k/OF800/data_clim/OF800_his_merged.nc')
ds_of800_bry = xr.open_dataset('/cluster/projects/nn9297k/OF800/data_bry/OF800_bry.nc')

In [ ]:
ds_of160_grid

In [ ]:
ds_of800_clim

### Interpolate (regrid) temperature from 160 m to 800 m

Prepare variables and resample from 160 to 800 meters domain.
`https://xesmf.readthedocs.io/en/latest/notebooks/Curvilinear_grid.html`.

In [ ]:
parameter_name = 'ubar'
lon_name, lat_name = 'lon_u', 'lat_u'

In [ ]:
da = regrid(ds_of160_grid,
            ds_of800_clim.isel(ocean_time=slice(0, 10)),
            parameter_name,
            lon_name,
            lat_name
            )

In [ ]:
try:
    da.isel(ocean_time=0, s_rho=-1).plot(figsize=(14, 7))
except ValueError:
    da.isel(ocean_time=0).plot(figsize=(14, 7))

In [ ]:
eta_name, xi_name, mask_name = 'eta_u', 'xi_u', 'mask_u'

In [ ]:
da = da.interpolate_na(dim=eta_name, method="nearest", fill_value="extrapolate")
da = da.interpolate_na(dim=xi_name, method="nearest", fill_value="extrapolate")
da = ds_of160_grid[mask_name] * da

In [ ]:
try:
    da.isel(ocean_time=0, s_rho=-1).plot(figsize=(14, 7))
except ValueError:
    da.isel(ocean_time=0).plot(figsize=(14, 7))

In [ ]:
try:
    ds_of800_clim[parameter_name].isel(ocean_time=0, s_rho=-1).plot(figsize=(14, 7))
except ValueError:
    ds_of800_clim[parameter_name].isel(ocean_time=0).plot(figsize=(14, 7))

In [ ]:
da

In [ ]:
ds = xr.Dataset({
    parameter_name: da
})

In [ ]:
ds

In [ ]:
ds.to_netcdf(f'/cluster/projects/nn9297k/OF160/Clm/{0:03d}_OF160_clm_{parameter_name}.nc')